In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [ ]:
class SeismicSet(Dataset):
    def __init__(self, img_path, sigma, transforms=None):
        self.img_path = img_path
        self.transforms = transforms
        self.imgs = os.listdir(img_path)
        self.sigma = sigma

    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        path_file = os.path.join(self.img_path, self.imgs[idx])
        clean = Image.open(path_file).convert("RGB")
        
        transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize images to 256x256
            transforms.ToTensor()  # Convert images to tensors
        ])

        clean = transform(clean)
        noise = torch.randn(clean.shape) * self.sigma
        noisy = clean + noise
        
        return noisy, clean


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # [B, 64, 128, 128]
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # [B, 128, 64, 64]
            nn.ReLU(True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # [B, 256, 32, 32]
            nn.ReLU(True),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),  # [B, 512, 16, 16]
            nn.ReLU(True),
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),  # [B, 256, 32, 32]
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  # [B, 128, 64, 64]
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # [B, 64, 128, 128]
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # [B, 3, 256, 256]
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
# Hyperparameters
num_epochs = 20
learning_rate = 0.001
batch_size = 16
sigma = 0.1
img_path = 'D:\\synthetic' # replace with your dataset path

# Dataset and Dataloader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = SeismicSet(img_path=img_path, sigma=sigma, transforms=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, Loss, and Optimizer
model = Autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    for data in dataloader:
        noisy_imgs, clean_imgs = data
        noisy_imgs = noisy_imgs.cuda()
        clean_imgs = clean_imgs.cuda()

        # Forward pass
        outputs = model(noisy_imgs)
        loss = criterion(outputs, clean_imgs)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training complete.')
